In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torchvision import transforms, models, datasets
import torchvision
import numpy as np
import h5py
from matplotlib import pyplot as plt
from skimage.transform import resize
from Utils import save_large_dataset, load_large_dataset, calculate_metric
plt.ion()   # interactive mode

In [2]:
CUDA = True

In [3]:
X = load_large_dataset('images')
Y = load_large_dataset('labels')

In [4]:
X = X.squeeze() #remove unnecessary dimension
Y = Y.squeeze()

In [ ]:
fig = plt.figure(figsize=(4,4))
my_slice = X[1005,:,90,:]
plt.imshow(my_slice, cmap="Greys")

In [5]:
X = X.squeeze() #remove unnecessary dimension
Y = Y.squeeze()

X = X[:,56:59,:,:] #take only 3 slices and treat them as channels
#X = X[:,:,90:93,:] #take only 3 slices and treat them as channels

print (X.shape)
print (Y.shape)

(1792, 3, 145, 121)
(1792,)


In [6]:
X_padded = np.pad(X,((0,0),(0,0),(40,39),(51,52)), 'constant') #pad with zeros to get 224x224 dimension of images
X = X_padded
X = np.float32(X)

In [7]:
X = np.rollaxis(X, 3, 2) #move channel dimension to be the first one
print (X.shape)

(1792, 3, 224, 224)


In [8]:
np.random.seed(8) #seed fixed for reproducibility
mask = np.random.rand(len(X)) < 0.9  #array of boolean variables

training_set = X[mask]
training_labels = Y[mask]

validation_set = X[~mask]
validation_labels = Y[~mask]

In [9]:
BATCH_SIZE = 32

In [10]:
training_set = torch.from_numpy(training_set) #convert to torch tensor
training_labels = torch.from_numpy(training_labels) #convert to torch tensor

In [11]:
validation_set = torch.from_numpy(validation_set) #convert to torch tensor
validation_labels = torch.from_numpy(validation_labels) #convert to torch tensor

In [12]:
training_labels = training_labels.long()

In [13]:
validation_labels = validation_labels.long()

In [14]:
print (training_labels.shape)
print (validation_labels.shape)

torch.Size([1624])
torch.Size([168])


In [15]:
dataset = torch.utils.data.TensorDataset(training_set, training_labels)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [16]:
val_set = torch.utils.data.TensorDataset(validation_set, validation_labels)
validation_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

# Model definition


In [17]:
# RESNET_18 model, with pretrained weights and fine-tuned to 2 classes
#net = models.resnet18(pretrained=True)
#num_ftrs = net.fc.in_features
#net.fc = nn.Linear(num_ftrs, 2)

# VGG_11 model with pretrained weights and fine-tuned to 2 classes
net = models.vgg11_bn(pretrained=True)
net.classifier._modules['6'] = nn.Linear(4096, 2)

criterion = nn.CrossEntropyLoss()

In [18]:
if (CUDA):
    net.cuda()

In [19]:
optimizer = optim.Adam(net.parameters(), lr=0.0001, weight_decay=1e-3)

# Training

In [20]:
for epoch in range(10):  
    
    net.train()
    running_loss = 0.0
    metric_sum = 0.0
   
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        if (CUDA):
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
       
        loss.backward()
        optimizer.step()

        # statistics
        running_loss += loss.data[0] * inputs.size(0)
        metric_sum += calculate_metric(preds,labels.data) * inputs.size(0)

    epoch_loss = running_loss / len(training_set)
    metric = metric_sum / len(training_set)
    print('Loss: {:.4f} Metric: {:.4f}'.format(epoch_loss, metric))

    #VALIDATION
    running_loss = 0.0
    metric_sum = 0.0
    net.eval() #set in the evaluation mode (important for Dropout and Batchnorm)
    
    for j, val_data in enumerate(validation_loader, 0):
        # get the inputs
        val_inputs, val_labels = val_data

        # wrap them in Variable
        if (CUDA):
            val_inputs, val_labels = Variable(val_inputs.cuda()), Variable(val_labels.cuda())
        else:
            val_inputs, val_labels = Variable(val_inputs), Variable(val_labels)

        outputs = net(val_inputs)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, val_labels)

        # statistics
        running_loss += loss.data[0] * val_inputs.size(0)
        metric_sum += calculate_metric(preds,val_labels.data) * val_inputs.size(0)

    val_epoch_loss = running_loss / len(validation_set)
    val_metric = metric_sum / len(validation_set)
    print('Validation Loss: {:.4f} Validation Metric: {:.4f}'.format(val_epoch_loss, val_metric))
    
print('Finished Training')

Loss: 0.6716 Metric: 0.5626
Validation Loss: 0.7004 Validation Metric: 0.5302
Loss: 0.6381 Metric: 0.6201
Validation Loss: 0.7039 Validation Metric: 0.5190
Loss: 0.5422 Metric: 0.7170
Validation Loss: 0.8451 Validation Metric: 0.5321
Loss: 0.3983 Metric: 0.8165
Validation Loss: 0.9632 Validation Metric: 0.5656
Loss: 0.2186 Metric: 0.9178
Validation Loss: 1.4337 Validation Metric: 0.5405
Loss: 0.1125 Metric: 0.9666
Validation Loss: 1.8605 Validation Metric: 0.4964
Loss: 0.1049 Metric: 0.9650
Validation Loss: 1.6741 Validation Metric: 0.5515
Loss: 0.0907 Metric: 0.9693
Validation Loss: 2.2463 Validation Metric: 0.4774
Loss: 0.0527 Metric: 0.9800
Validation Loss: 2.3506 Validation Metric: 0.5204
Loss: 0.0738 Metric: 0.9742
Validation Loss: 1.9342 Validation Metric: 0.5113
Finished Training


# Traininig on augmented set

In [21]:
X = load_large_dataset('new_augmented_train_set')
Y = load_large_dataset('new_augmented_train_set_labels')

In [22]:
print (X.shape)
print (Y.shape)

(8120, 3, 121, 145)
(8120,)


In [23]:
X_padded = np.pad(X,((0,0),(0,0),(51,52),(40,39)), 'constant') #pad with zeros to get 224x224 dimension of images
X = X_padded
X = np.float32(X)

In [24]:
training_set = torch.from_numpy(X) #convert to torch tensor
training_labels = torch.from_numpy(Y) #convert to torch tensor

In [25]:
training_labels = training_labels.long()

In [26]:
dataset = torch.utils.data.TensorDataset(training_set, training_labels)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Model definition

In [27]:
# RESNET_18 model, with pretrained weights and fine-tuned to 2 classes
#net = models.resnet18(pretrained=True)
#num_ftrs = net.fc.in_features
#net.fc = nn.Linear(num_ftrs, 2)

# VGG_11 model with pretrained weights and fine-tuned to 2 classes
net = models.vgg11_bn(pretrained=True)
net.classifier._modules['6'] = nn.Linear(4096, 2)

criterion = nn.CrossEntropyLoss()

In [28]:
if (CUDA):
    net.cuda()

In [29]:
optimizer = optim.Adam(net.parameters(), lr=0.0001, weight_decay=1e-3)

In [30]:
for epoch in range(10):  
    
    net.train()
    running_loss = 0.0
    metric_sum = 0.0
   
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        if (CUDA):
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
       
        loss.backward()
        optimizer.step()

        # statistics
        running_loss += loss.data[0] * inputs.size(0)
        metric_sum += calculate_metric(preds,labels.data) * inputs.size(0)

    epoch_loss = running_loss / len(training_set)
    metric = metric_sum / len(training_set)
    print('Loss: {:.4f} Metric: {:.4f}'.format(epoch_loss, metric))

    #VALIDATION
    running_loss = 0.0
    metric_sum = 0.0
    net.eval() #set in the evaluation mode (important for Dropout and Batchnorm)
    
    for j, val_data in enumerate(validation_loader, 0):
        # get the inputs
        val_inputs, val_labels = val_data

        # wrap them in Variable
        if (CUDA):
            val_inputs, val_labels = Variable(val_inputs.cuda()), Variable(val_labels.cuda())
        else:
            val_inputs, val_labels = Variable(val_inputs), Variable(val_labels)

        outputs = net(val_inputs)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, val_labels)

        # statistics
        running_loss += loss.data[0] * val_inputs.size(0)
        metric_sum += calculate_metric(preds,val_labels.data) * val_inputs.size(0)

    val_epoch_loss = running_loss / len(validation_set)
    val_metric = metric_sum / len(validation_set)
    print('Validation Loss: {:.4f} Validation Metric: {:.4f}'.format(val_epoch_loss, val_metric))
    
print('Finished Training')

Loss: 0.6537 Metric: 0.6044
Validation Loss: 0.6697 Validation Metric: 0.4961
Loss: 0.6442 Metric: 0.6127
Validation Loss: 0.7085 Validation Metric: 0.5073
Loss: 0.6411 Metric: 0.6170
Validation Loss: 0.7053 Validation Metric: 0.5157
Loss: 0.6420 Metric: 0.6169
Validation Loss: 0.6934 Validation Metric: 0.5360
Loss: 0.6439 Metric: 0.6232
Validation Loss: 0.7442 Validation Metric: 0.4861
Loss: 0.6405 Metric: 0.6222
Validation Loss: 0.6966 Validation Metric: 0.4823
Loss: 0.6382 Metric: 0.6177
Validation Loss: 1.4280 Validation Metric: 0.5000
Loss: 0.6375 Metric: 0.6251
Validation Loss: 0.6919 Validation Metric: 0.5107
Loss: 0.6370 Metric: 0.6218
Validation Loss: 0.7447 Validation Metric: 0.4807
Loss: 0.6385 Metric: 0.6138
Validation Loss: 0.7008 Validation Metric: 0.4850
Finished Training


With augmented data set convergence is slower for training set, unfortunately it does not help for generalization as metrics for the validation set do not improve.